In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.io as pio
pio.renderers.default = "vscode"


plt.style.use('default')
%matplotlib inline

In [2]:

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn import metrics
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
from numpy import array
from keras import optimizers
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error


In [3]:
#read data
df = pd.read_csv('clean_data.csv')
df.dropna(inplace=True)
df['SAMPLING DATE']=pd.to_datetime(df['SAMPLING DATE'])
df.set_index('SAMPLING DATE', inplace=True)
df.sort_index(ascending=True, inplace=True)
df.drop(['City'], axis=1, inplace=True)
df_copy = df.copy()

In [4]:
df.head()

,SO2 µg/m3,NO2 µg/m3,PM10 µg/m3,PM2.5 µg/m3,AQI
SAMPLING DATE,,,,,
2017-01-06,7.9,12.5,31.9,10.4,32.0
2017-01-08,13.0,11.0,34.0,23.0,34.0
2017-01-08,7.4,11.3,34.8,11.2,35.0
2017-01-09,8.2,11.8,44.4,14.8,44.0
2017-02-05,2.0,11.0,124.0,23.0,124.0


In [5]:
#df.reset_index(inplace=True)
X = df[['PM10 µg/m3']] 
y = df['AQI']

In [6]:
X.head(5)

,PM10 µg/m3
SAMPLING DATE,
2017-01-06,31.9
2017-01-08,34.0
2017-01-08,34.8
2017-01-09,44.4
2017-02-05,124.0


In [7]:
y.head(5)

SAMPLING DATE
2017-01-06     32.0
2017-01-08     34.0
2017-01-08     35.0
2017-01-09     44.0
2017-02-05    124.0
Name: AQI, dtype: float64

In [8]:
y.info()

<class 'pandas.core.series.Series'>
DatetimeIndex: 1180 entries, 2017-01-06 to 2021-12-12
Series name: AQI
Non-Null Count  Dtype  
--------------  -----  
1180 non-null   float64
dtypes: float64(1)
memory usage: 18.4 KB


In [9]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=15)
X_train, X_test= np.split(X, [int(.8 *len(X))])
y_train, y_test= np.split(y, [int(.8 *len(y))])


In [10]:
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)

LinearRegression()

In [11]:
pred = lin_reg.predict(X_test)

print(f"Mean Abs Error: {metrics.mean_absolute_error(y_test, pred)}")
print(f"Mean Sq Error: {metrics.mean_squared_error(y_test, pred)}")
print(f"Root Mean Error: {np.sqrt(metrics.mean_squared_error(y_test, pred))}")

Mean Abs Error: 15.625578995984917
Mean Sq Error: 1131.4221074649508
Root Mean Error: 33.636618549802996


In [12]:
results = pd.DataFrame(pred, columns=['LinearRegression'])
results.index = X_test.index


In [13]:
results

,LinearRegression
SAMPLING DATE,
2020-10-07,47.221082
2020-10-07,38.236969
2020-10-07,13.734843
2020-10-09,13.734843
2020-10-10,54.571719
...,...
2021-12-02,113.376820
2021-12-03,85.607745
2021-12-03,68.456257


In [14]:
from sklearn.preprocessing import PolynomialFeatures  
poly_regs = PolynomialFeatures(degree= 10)  
x_train_poly = poly_regs.fit_transform(X_train)  
poly_lin_reg = LinearRegression()  
poly_lin_reg.fit(x_train_poly, y_train)  

LinearRegression()

In [15]:
x_test_poly = poly_regs.fit_transform(X_test)  
pred = poly_lin_reg.predict(x_test_poly)
results['Poly_Reg']=pred

print(f"Mean Abs Error: {metrics.mean_absolute_error(y_test, pred)}")
print(f"Mean Sq Error: {metrics.mean_squared_error(y_test, pred)}")
print(f"Root Mean Error: {np.sqrt(metrics.mean_squared_error(y_test, pred))}")

Mean Abs Error: 12.55541124363736
Mean Sq Error: 690.3282786127626
Root Mean Error: 26.27409900667885


In [16]:
regressor=SVR(kernel='rbf', C=10)
regressor.fit(X_train, y_train)

SVR(C=10)

In [17]:
pred = regressor.predict(X_test)
results['SVR_10']=pred

In [18]:
print(f"Mean Abs Error: {metrics.mean_absolute_error(y_test, pred)}")
print(f"Mean Sq Error: {metrics.mean_squared_error(y_test, pred)}")
print(f"Root Mean Error: {np.sqrt(metrics.mean_squared_error(y_test, pred))}")

Mean Abs Error: 10.968530743841542
Mean Sq Error: 724.4788432025257
Root Mean Error: 26.916144657111012


In [19]:
regressor=SVR(kernel='rbf', C=50)
regressor.fit(X_train, y_train)

SVR(C=50)

In [20]:
pred = regressor.predict(X_test)
results['SVR_50']=pred

In [21]:
print(f"Mean Abs Error: {metrics.mean_absolute_error(y_test, pred)}")
print(f"Mean Sq Error: {metrics.mean_squared_error(y_test, pred)}")
print(f"Root Mean Error: {np.sqrt(metrics.mean_squared_error(y_test, pred))}")

Mean Abs Error: 10.815459530434804
Mean Sq Error: 708.550101450339
Root Mean Error: 26.6186044234167


In [22]:
fig = go.Figure()
fig.update_layout(title="Comparison of prediction output with data")

fig.add_trace(go.Scatter(x=X['PM10 µg/m3'], y=y, name="Original data",  mode='markers'))

fig.add_trace(go.Scatter(x=X_test['PM10 µg/m3'], y=results['SVR_10'], name="SVR(c=10)", mode='markers'))

fig.add_trace(go.Scatter(x=X_test['PM10 µg/m3'], y=results['SVR_50'], name="SVR(c=50)", mode='markers'))

fig.add_trace(go.Scatter(x=X_test['PM10 µg/m3'], y=results['LinearRegression'], name="Linear Regression"))

fig.add_trace(go.Scatter(x=X_test['PM10 µg/m3'], y=results['Poly_Reg'], name="Polynomial Regression", mode='markers'))


fig.show()


In [23]:
def create_dataset(dataset, look_back=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-look_back-1):
		a = dataset[i:(i+look_back), 0]
		dataX.append(a)
		dataY.append(dataset[i + look_back, 0])
	return np.array(dataX), np.array(dataY)

	#dataX -> 100 200 300 Y -> 400 
	#dataY -> 200 300 400 Y -> 500 

In [24]:
import tensorflow as tf
tf.random.set_seed(7)

In [25]:
dataframe = pd.read_csv('clean_data.csv', usecols=[5], engine='python')
dataset = dataframe.values
dataset = dataset.astype('float32')

In [26]:
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)

In [27]:
train_size = int(len(dataset) * 0.67)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]

In [28]:
look_back = 3
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

In [29]:
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

In [30]:
# model = Sequential()
# model.add(LSTM(4, input_shape=(1, look_back)))
# model.add(Dense(1))
# model.compile(loss='mean_squared_error', optimizer='adam')
# model.fit(trainX, trainY, epochs=10, batch_size=1, verbose=2)

In [31]:
from tensorflow import keras
model = keras.models.load_model('AQI_LSTM.h5')

In [32]:
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)


15/15 [==============================] - 0s 7ms/step


In [33]:
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])
testPredict = scaler.inverse_transform(testPredict)
testY1 = scaler.inverse_transform([testY])

In [34]:
print(f"Mean Abs Error: {metrics.mean_absolute_error(testY1[0], testPredict[:,0])}")
print(f"Mean Sq Error: {metrics.mean_squared_error(testY1[0], testPredict[:,0])}")
print(f"Root Mean Error: {np.sqrt(metrics.mean_squared_error(testY1[0], testPredict[:,0]))}")

Mean Abs Error: 14.568286552699073
Mean Sq Error: 420.34122893641717
Root Mean Error: 20.502224975265907


In [35]:
# trainPredictPlot = np.empty_like(dataset)
# trainPredictPlot[:, :] = np.nan
# trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict

# testPredictPlot = np.empty_like(dataset)
# testPredictPlot[:, :] = np.nan
# testPredictPlot[len(trainPredict)+(look_back*2)+1:len(dataset)-1, :] = testPredict

In [36]:
# plt.plot(scaler.inverse_transform(dataset))
# plt.plot(trainPredictPlot)
# plt.plot(testPredictPlot)
# plt.show()

In [37]:
data = pd.read_csv('clean_data.csv', index_col=0)
data = data.iloc[train_size+look_back+1:,:]
data.sort_index(ascending=True, inplace=True)

series_results = pd.DataFrame(testPredict, columns=['LSTM'])
series_results.index = data.index
series_results.sort_index(ascending=True, inplace=True)

#print(series_results.head(400))

In [38]:
from statsmodels.tsa.arima.model import ARIMA

In [39]:
history = [x for x in train]

In [40]:
predictions = list()

In [41]:
predictions.clear()
for t in range(len(test)):
	model = ARIMA(history, order=(1,1,1))
	model_fit = model.fit()
	output = model_fit.forecast()
	yhat = output[0]
	predictions.append(yhat)
	obs = test[t]
	history.append(obs)
	print('predicted=%f, expected=%f' % (yhat, obs))

predicted=0.168476, expected=0.184211


C:\Users\Tejas\AppData\Local\Programs\Python\Python39\lib\site-packages\statsmodels\base\model.py:604: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals



predicted=0.170054, expected=0.164474
predicted=0.166958, expected=0.177632
predicted=0.170436, expected=0.184211
predicted=0.173888, expected=0.180921
predicted=0.174993, expected=0.164474
predicted=0.171226, expected=0.171053
predicted=0.171844, expected=0.213816
predicted=0.184594, expected=0.200658
predicted=0.186555, expected=0.184211
predicted=0.184451, expected=0.164474
predicted=0.178440, expected=0.177632
predicted=0.179579, expected=0.184211
predicted=0.181171, expected=0.180921
predicted=0.180794, expected=0.243421
predicted=0.199689, expected=0.263158
predicted=0.214548, expected=0.351974
predicted=0.251428, expected=0.276316
predicted=0.248978, expected=0.361842
predicted=0.280745, expected=0.338816
predicted=0.290524, expected=0.351974
predicted=0.304646, expected=0.335526
predicted=0.309496, expected=0.342105
predicted=0.316974, expected=0.338816
predicted=0.321225, expected=0.266447
predicted=0.302691, expected=0.319079
predicted=0.311269, expected=0.335526
predicted=0.

C:\Users\Tejas\AppData\Local\Programs\Python\Python39\lib\site-packages\statsmodels\base\model.py:604: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals



predicted=0.112498, expected=0.115132
predicted=0.113062, expected=0.111842
predicted=0.112483, expected=0.134868
predicted=0.119504, expected=0.128289
predicted=0.120748, expected=0.131579
predicted=0.123385, expected=0.118421
predicted=0.121049, expected=0.121711
predicted=0.121509, expected=0.125000
predicted=0.122575, expected=0.131579
predicted=0.125139, expected=0.121711
predicted=0.123450, expected=0.131579
predicted=0.126146, expected=0.154605
predicted=0.134468, expected=0.138158
predicted=0.133667, expected=0.148026
predicted=0.137698, expected=0.138158
predicted=0.136841, expected=0.144737
predicted=0.139169, expected=0.164474
predicted=0.146500, expected=0.157895
predicted=0.148306, expected=0.148026
predicted=0.147292, expected=0.167763
predicted=0.153588, expected=0.171053
predicted=0.157652, expected=0.157895
predicted=0.156431, expected=0.180921
predicted=0.163909, expected=0.213816
predicted=0.177797, expected=0.194079
predicted=0.179389, expected=0.180921
predicted=0.

C:\Users\Tejas\AppData\Local\Programs\Python\Python39\lib\site-packages\statsmodels\base\model.py:604: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals



predicted=0.175382, expected=0.154605
predicted=0.172216, expected=0.075658
predicted=0.143637, expected=0.088816
predicted=0.133071, expected=0.154605
predicted=0.144223, expected=0.213816
predicted=0.165128, expected=0.055921
predicted=0.126040, expected=0.049342
predicted=0.108804, expected=0.032895
predicted=0.090717, expected=0.016447
predicted=0.072907, expected=0.213816
predicted=0.123016, expected=0.072368
predicted=0.098104, expected=0.062500
predicted=0.089443, expected=0.029605
predicted=0.073277, expected=0.020724
predicted=0.061008, expected=0.154605
predicted=0.094378, expected=0.059211
predicted=0.077454, expected=0.069079
predicted=0.076602, expected=0.154605
predicted=0.101777, expected=0.078947
predicted=0.089485, expected=0.105263
predicted=0.095448, expected=0.013158
predicted=0.068744, expected=0.121711
predicted=0.090690, expected=0.108553
predicted=0.093279, expected=0.098684
predicted=0.093492, expected=0.134868
predicted=0.105926, expected=0.072368
predicted=0.

C:\Users\Tejas\AppData\Local\Programs\Python\Python39\lib\site-packages\statsmodels\base\model.py:604: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals



predicted=0.115488, expected=0.032895
predicted=0.090505, expected=0.055921
predicted=0.085045, expected=0.009868
predicted=0.063609, expected=0.171053
predicted=0.103666, expected=0.197368
predicted=0.127314, expected=0.266447
predicted=0.165400, expected=0.217105
predicted=0.171883, expected=0.062500
predicted=0.131549, expected=0.000000
predicted=0.095738, expected=0.171053
predicted=0.130381, expected=0.105263
predicted=0.117951, expected=0.164474
predicted=0.134390, expected=0.151316
predicted=0.136739, expected=0.101974
predicted=0.123919, expected=0.134868
predicted=0.129774, expected=0.095395
predicted=0.118076, expected=0.243421
predicted=0.161107, expected=0.138158
predicted=0.145089, expected=0.042763
predicted=0.112608, expected=0.253289
predicted=0.165340, expected=0.203947
predicted=0.168788, expected=0.177632
predicted=0.168017, expected=0.184211
predicted=0.172270, expected=0.138158
predicted=0.159993, expected=0.190789
predicted=0.172212, expected=0.263158
predicted=0.

In [42]:
normal = [[val] for val in predictions]
testPredict = scaler.inverse_transform(normal[:-4])
testY2 = scaler.inverse_transform([testY])

In [43]:
print(len(testY2[0]))
print(len(testPredict))

450
450


In [44]:
print(f"Mean Abs Error: {metrics.mean_absolute_error(testY2[0], testPredict[:,0])}")
print(f"Mean Sq Error: {metrics.mean_squared_error(testY2[0], testPredict[:,0])}")
print(f"Root Mean Error: {np.sqrt(metrics.mean_squared_error(testY2[0], testPredict[:,0]))}")

Mean Abs Error: 17.81890581794742
Mean Sq Error: 619.9335922393743
Root Mean Error: 24.898465660344904


In [45]:
series_results["ARIMA"] = testPredict
series_results.head()

,LSTM,ARIMA
SAMPLING DATE,,
2017-01-08,63.546577,61.216827
2017-02-05,63.803818,61.696517
2017-02-06,64.806122,60.755374
2017-02-11,63.386082,61.812459
2017-02-12,62.676838,62.861985


In [46]:
#df.reset_index(inplace=True)
X = df_copy.iloc[:,:-1]
y = df_copy.iloc[:,-1]

In [47]:
train_size = int(len(X)*.6)
X_train, y_train, X_test, y_test = df_copy.iloc[:train_size,:-1], df_copy.iloc[:train_size,-1], df_copy.iloc[train_size:,:-1], df_copy.iloc[:train_size,-1]

In [48]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [49]:
from sklearn.decomposition import PCA

pca = PCA()
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)

In [50]:
regressor=SVR(kernel='rbf', C=10)
regressor.fit(X_train, y_train)

SVR(C=10)

In [51]:
len(X_test)

472

In [52]:
pred = regressor.predict(X_test)
print(len(pred))
series_results['SVR_PCA']=pred[:len(series_results)]

472


In [53]:
print(f"Mean Abs Error: {metrics.mean_absolute_error(y_test[:len(pred)], pred)}")
print(f"Mean Sq Error: {metrics.mean_squared_error(y_test[:len(pred)], pred)}")
print(f"Root Mean Error: {np.sqrt(metrics.mean_squared_error(y_test[:len(pred)], pred))}")

Mean Abs Error: 28.467080907778165
Mean Sq Error: 1404.5319642477675
Root Mean Error: 37.47708585586355


In [58]:
fig = go.Figure()
fig.update_layout(
    title="Comparison of predicted AQI and Measured AQI",
    xaxis_title="Date",
    yaxis_title="AQI Level",
    legend_title="Legend",
)

fig.update_xaxes(range=['2016-10-01', '2022-03-01'])
##
fig.add_trace(go.Scatter(x=results.index, y=results['SVR_10'], name="SVR(c=10)",line_shape='spline'))

fig.add_trace(go.Scatter(x=results.index, y=results['LinearRegression'], name="Linear Regression"))

fig.add_trace(go.Scatter(x=results.index, y=results['Poly_Reg'], name="Polynomial Regression"))
##

fig.add_trace(go.Scatter(x=data.index, y=data['AQI'], name="Actual"))

fig.add_trace(go.Scatter(x=series_results.index, y=series_results['LSTM'], name="LSTM", line_shape='spline'))

fig.add_trace(go.Scatter(x=series_results.index, y=series_results['ARIMA'], name="ARIMA", line_shape='spline'))

fig.add_trace(go.Scatter(x=data.index, y=series_results['SVR_PCA'], name="SVR_PCA(c=10)",line_shape='spline'))

fig.add_trace(go.Scatter(x=data.index, y=data.AQI.rolling(14).mean(), name="MA(14)",line_shape='spline'))

fig.show()

In [55]:
data

,SO2 µg/m3,NO2 µg/m3,PM10 µg/m3,PM2.5 µg/m3,AQI,City
SAMPLING DATE,,,,,,
2017-01-08,13.0,11.0,34.0,23.0,34.0,Mapusa
2017-02-05,2.0,11.0,124.0,23.0,124.0,Mapusa
2017-02-06,2.0,7.0,33.0,23.0,33.0,Mapusa
2017-02-11,2.0,11.0,223.0,23.0,182.0,Mapusa
2017-02-12,5.0,10.0,28.0,23.0,28.0,Mapusa
...,...,...,...,...,...,...
2021-12-02,2.0,19.0,124.0,50.0,116.0,Mapusa
2021-12-02,9.0,16.0,77.0,33.0,77.0,Ponda
2021-12-03,9.0,16.0,69.0,34.0,69.0,Ponda


In [56]:
series_results

,LSTM,ARIMA,SVR_PCA
SAMPLING DATE,,,
2017-01-08,63.546577,61.216827,84.815579
2017-02-05,63.803818,61.696517,115.328046
2017-02-06,64.806122,60.755374,97.046105
2017-02-11,63.386082,61.812459,80.901292
2017-02-12,62.676838,62.861985,82.976940
...,...,...,...
2021-12-02,63.809189,75.995576,52.155095
2021-12-02,74.577370,70.924364,52.175014
2021-12-03,76.121971,79.442661,81.069188


In [57]:
results

,LinearRegression,Poly_Reg,SVR_10,SVR_50
SAMPLING DATE,,,,
2020-10-07,47.221082,42.146346,43.033707,43.127624
2020-10-07,38.236969,38.324292,31.820889,31.772812
2020-10-07,13.734843,36.675230,34.099968,35.224849
2020-10-09,13.734843,36.675230,34.099968,35.224849
2020-10-10,54.571719,47.823846,52.222935,52.122763
...,...,...,...,...
2021-12-02,113.376820,123.364409,116.285486,116.330202
2021-12-03,85.607745,92.723495,90.247644,90.194336
2021-12-03,68.456257,65.071102,68.829563,68.862788
